# Image Classification

Classification example using the **Quick Draw Dataset**
- Download the datasets from [here](https://console.cloud.google.com/storage/browser/quickdraw_dataset/full/numpy_bitmap?pli=1) and place them in the directory `.data`

In [4]:
import os

import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

##### Preprocess the dataset

In [8]:
data_path = '.data'
input_shape = (28, 28, 1)
data_files = []


for root, dirs, files, in os.walk(data_path):
    data_files += [os.path.join(root, name) for name in files]

In [ ]:
n_images = 100000
images_per_category = n_images // len(data_files)
data, targets = [], []

for i, file in enumerate(data_files):
    x = np.load(file) / 255.
    y = i * x.shape[0]
    
    x = x[:images_per_category]
    y = y[:images_per_category]
    

    data, targets = np.concatenate([data, x], axis=0), np.concatenate([targets, y], axis=0)


# splits
x_train, x_test, y_train, y_test = train_test_split(data, target,
                                                    train_size=0.75,
                                                    random_state=20)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train,
                                                     train_size=.9,
                                                     random_state=20)

##### define the model

In [9]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        
        inputs = tf.keras.Input(shape=input_shape)
        self.x0 = tf.keras.layers.Conv2D(filters=32, kernel_size=3,
                                         padding='valid', activation='leaky_relu',
                                        input_shape=input_shape)
        self.x1 = tf.keras.layers.MaxPool2D()
        self.x2 = tf.keras.layers.Dropout(0.4)
        
        self.x3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3,
                                         padding='valid', activation='leaky_relu')
        self.x4 = tf.keras.layers.MaxPool2D()
        self.x5 = tf.keras.layers.Dropout(0.4)
        
        self.x6 = tf.keras.layers.Flatten()
        self.x7 = tf.keras.layers.Dense(128, activation='leaky_relu')
        self.x8 = tf.keras.layers.Dropout(0.2)
        
        self.output_pred = tf.keras.layers.Dense(len(data_files), activation='softmax')
    
    def call(self, inputs):
        x = self.x0(inputs)
        
        for i in range(1, 10):
            x = getattr(self, f'x{i}')(x)
        
        return self.output_pred(x)

##### train the model

In [ ]:
epochs = 20
batch_size = 128
save_path = '~/.tensorboard/tf2/1'

model = Model()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
             metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size,
         callbacks=[tf.keras.callbacks.TensorBoard(save_path)],
         validation_data=(x_valid, y_valid))

model.evaluate(x_test, y_test)